In [54]:

%load_ext autoreload
%autoreload 2
from rioplatense_hs.openai import get_completion
from datasets import load_dataset

dataset = load_dataset("piuba-bigdata/contextualized_hate_speech")


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [55]:
import json

raw_dataset = json.load(open("../data/dataset.raw.json"))

type(raw_dataset)

list

In [11]:
dataset["dev"]

Dataset({
    features: ['id', 'title', 'text', 'context_tweet', 'HATEFUL', 'body', 'CALLS', 'WOMEN', 'LGBTI', 'RACISM', 'CLASS', 'POLITICS', 'DISABLED', 'APPEARANCE', 'CRIMINAL'],
    num_rows: 9106
})

In [12]:
from rioplatense_hs.preprocessing import labels, label_to_text

df_train = dataset["train"].to_pandas()
df_test = dataset["test"].to_pandas()
df_dev = dataset["dev"].to_pandas()
df_train["num_hatred"] = df_train[labels].sum(axis=1)
df_dev["num_hatred"] = df_dev[labels].sum(axis=1)
df_test["num_hatred"] = df_test[labels].sum(axis=1)

df_train["text_label"] = df_train.apply(lambda x: label_to_text(x), axis=1)
df_dev["text_label"] = df_dev.apply(lambda x: label_to_text(x), axis=1)
df_test["text_label"] = df_test.apply(lambda x: label_to_text(x), axis=1)


In [13]:
labels

['WOMEN',
 'LGBTI',
 'RACISM',
 'CLASS',
 'POLITICS',
 'DISABLED',
 'APPEARANCE',
 'CRIMINAL']

In [14]:
import pandas as pd

pd.options.display.max_colwidth = 1000
df_train.loc[df_train["LGBTI"] > 0, ["title", "text", "text_label"]].sample(20)

,title,text,text_label
8021,"Quién es Ritchie Torres, el primer afrolatino homosexual elegido para el Congreso de EEUU","@usuario Negro, puto y zurdo. Esos son los méritos del tipo este. Es un congresista puesto por el establishment.","lgbti, racismo, política"
27278,Coronavirus en la Argentina: fue a donar plasma tras recuperarse pero se lo impidieron por ser gay,"@usuario Si sos gay no podés donar, si declarás que sos promiscuo no podés donar,si usaste drogas no podés donar, si te tatuaste hace menos de 6 meses no podés donar. ¿De quién es la culpa, del que dice la verdad al momento de desear hacerlo, o de los protocolos retrógrados que tenemos?",lgbti
22575,"Ricky Martin: ""Soy un hombre latino y homosexual viviendo en los Estados Unidos, soy una amenaza""","@usuario @usuario el problema no es que seas latino y trolo, el problema es que en el peor de los casos sos un lobbista hijo de mil puta y en el mejor de los casos sos un imbecil mononeuronal, o a ver si te atreves a declarar lo mismo en paises con ley sharia, cobarde sin verguenza","lgbti, racismo, discapacidad"
21617,"Pibita trans, activista negra, ni hombre ni mujer: ""Mi cuerpo es mi resistencia""",@usuario Pibita trans=nene,lgbti
22514,"Ricky Martin: ""Soy un hombre latino y homosexual viviendo en los Estados Unidos, soy una amenaza""","@usuario @usuario Proba con ir a triunfar a un país árabe, a ver qué tal te va, MARICON",lgbti
22559,"Pablo Duggan, sobre su casamiento: ""Karen es la mujer de mi vida, estoy muy enamorado""","@usuario Ah, también panquequea el toor...","mujer, lgbti"
8648,Luis Novaresio le dedicó un romántico mensaje a Braulio Bauab por su cumpleaños,@usuario quien será la pasiva?,lgbti
31588,"El llanto de Gustavo López tras recuperarse del coronavirus: ""La incertidumbre de no saber si salís es muy fuerte""",@usuario NO SEAS TROLO MAN,lgbti
28223,"Las lágrimas de Guillermo Andino tras recuperarse de coronavirus: ""El miedo es no poder contarla""","@usuario No seas trolo, man.",lgbti
27933,"Tras los comentarios de J.K. Rowling, Emma Watson defendió al colectivo trans",@usuario No veo fallas en la logica de JK para que salte el come trava de 22 pesos de Harry y la sucia inmunda. Va a salir el colorado a decir q se garcho un travesti y le gusto,"lgbti, apariencia"


In [61]:
from rioplatense_hs.prompting import build_prompt

print(build_prompt(contexto="Estados Unidos restringe los vuelos debido al COVID-19", texto="Todo esto es una locura"))

Determinar si el siguiente mensaje contiene discurso discriminatorio o no. Entendemos que hay discurso discriminatorio en el tweet si tiene declaraciones de carácter intenso e irracional de rechazo, enemistad y aborrecimiento contra un individuo o contra un grupo, siendo estos objetivos de estas expresiones por poseer una característica protegida. Las características protegidas que contemplamos son:

mujer: Mujeres
lgbti: Género, identidad sexual
racismo: Inmigrantes, extranjeros, religión, pueblos aborígenes u otras nacionalidades
clase: Clase, situación socioeconómica o barrio de residencia
discapacidad: Discapacidades, problemas de salud mental o de adicción al alcohol, drogas u otros estupefacientes
política: Opinión o ideología política
aspecto: Aspecto o edad
criminal: Antecedentes penales o estar privado de la libertad

En contexto tendremos un titular de una noticia y en texto el comentario a analizar. Responder una o varias de las características separadas por coma, o bien "na

In [18]:
# sample_df = df_test.sample(100)

sample_df = df_dev.sample(700, random_state=2022)

In [19]:
(sample_df["num_hatred"] > 0).mean()

0.16857142857142857

In [20]:
outs = {

}

In [21]:
print(template_prompt)

Determinar si el siguiente mensaje contiene discurso discriminatorio o no. Entendemos que hay discurso discriminatorio en el tweet si tiene declaraciones de carácter intenso e irracional de rechazo, enemistad y aborrecimiento contra un individuo o contra un grupo, siendo estos objetivos de estas expresiones por poseer una característica protegida. Las características protegidas que contemplamos son:

mujer: Mujeres
lgbti: Género, identidad sexual
racismo: Inmigrantes, extranjeros, religión, pueblos aborígenes u otras nacionalidades
clase: Clase, situación socioeconómica o barrio de residencia
discapacidad: Discapacidades, problemas de salud mental o de adicción al alcohol, drogas u otros estupefacientes
política: Opinión o ideología política
aspecto: Aspecto o edad
criminal: Antecedentes penales o estar privado de la libertad

En contexto tendremos un titular de una noticia, y en texto el comentario a analizar. Responder una o varias de las características separadas por coma, o bien "n

In [23]:
from tqdm import tqdm
import time
from rioplatense_hs.preprocessing import preprocess_tweet
from pysentimiento.preprocessing import preprocess_tweet as _pysent_preprocess


for idx, row in tqdm(sample_df.iterrows(), total=sample_df.shape[0]):

    if idx in outs:
        continue

    contexto = _pysent_preprocess(row["context_tweet"], preprocess_hashtags=False, demoji=False, preprocess_handles=False)
    texto = preprocess_tweet(row["text"])
    prompt = template_prompt + f"contexto: {contexto}\ntexto: {texto}\nsalida: "

    response = get_completion(prompt)

    outs[idx] = response.choices[0].message.content

    time.sleep(0.025)

  0%|          | 0/700 [00:00<?, ?it/s]

100%|██████████| 700/700 [05:33<00:00,  2.10it/s] 


In [ ]:
import json

with open("outputs.json", "w") as f:
    json.dump(outs, f)


In [ ]:
labels = []

In [ ]:
from collections import Counter

counter = Counter()


for v in outs.values():
    for cl in v.split(","):
        counter[cl.strip().lower()] += 1


In [ ]:

counter

In [24]:
sample_df["pred_cot"] = sample_df.index.map(outs)


In [25]:
sample_df[["context_tweet", "text", "text_label","pred_cot"]]

,context_tweet,text,text_label,pred_cot
6934,Coronavirus: un diario inglés asegura que los mercados chinos siguen vendiendo murciélagos y carne de animales domésticos https://t.co/qFPs4aHf6D,@usuario Que gente de mierda .,nada,"El texto no contiene ningún tipo de referencia a características protegidas ni expresiones discriminatorias. La respuesta final es ""nada""."
5708,"Murió el periodista Mario Pereyra, ícono de la radio de Córdoba https://t.co/MZtxmqFP4o",@usuario Cuánto zurdito resentido en los comentarios...,política,"El texto utiliza el término ""zurdito"" de manera despectiva para referirse a personas de ideología política izquierdista. La respuesta final es ""política""."
7287,"""Se terminó, a salir a la calle”: detuvieron a un hombre que se grabó llamando a romper la cuarentena obligatoria https://t.co/VCK2uezc9p",@usuario Flor de imbécil,nada,"El texto contiene un insulto, pero no se dirige a ninguna característica protegida. La respuesta final es ""nada""."
5575,El polémico mensaje de Cinthia Fernández sobre la muerte de un delincuente en un intento de robo https://t.co/odv7PghHxL,@usuario Y que tiene tiene de polémico???,nada,"El texto no contiene discurso discriminatorio. La respuesta final es ""nada""."
6454,Coronavirus: China anunció que no registró ningún caso nuevo local https://t.co/bg8d1fSwBX https://t.co/M47osbSpVL,@usuario Chinos de mierda q se maten todos,racismo,"Desea que se maten los chinos por ser los supuestos culpables del COVID-19. La respuesta final es ""racismo"""
...,...,...,...,...
3204,"El contundente mensaje de Noelia Marzol sobre el feminismo y el sexo: ""Queremos coger en libertad"" https://t.co/Quo3h68jdU",@usuario Si.lo.hacen,nada,"No se menciona ninguna característica protegida en el texto. La respuesta final es ""nada""."
102,Alberto Fernández: “La pandemia finalmente nos dejó más muertos de los que creíamos tener” https://t.co/0T7hXsGS65,"@usuario baja el dedito , goberna y deja de gritarnos",nada,"El texto contiene una crítica hacia Alberto Fernández, pero no se basa en ninguna de las características protegidas mencionadas. La respuesta final es ""nada""."
2061,"Tras las críticas del kirchnerismo duro, Alberto Fernández llamará a Maduro para explicarle la condena argentina en la ONU | Por Román Lejtman https://t.co/yYnNvu50oO","@usuario ""Entendeme, Nicolás. Vos sos un poco brusco con los ciudadanos. Si lo taparas un poco tal vez, pero así de obvio no se puede""",nada,"El texto no contiene discurso discriminatorio, solamente expresa una opinión sobre la forma de actuar de Nicolás Maduro. La respuesta final es ""nada""."
8457,Los mapuches avanzaron en la toma de tierras en la Patagonia: cada hectárea está valuada en USD 100 mil https://t.co/52gdoBivK6,"@usuario Que los caguen a balazos y los echen como perros.\nUn tipo que labura todo el día y aporta al Estado no puede adquirir un triste terreno, y si con suerte lo adquiere, tiene que solicitar un crédito o préstamo para poder construir.",racismo,"El texto desea la violencia y discriminación hacia los mapuches por su condición de pertenecer a un pueblo aborigen. Además, se refiere a su condición de ""perros"" y alude a su situación socioeconómica. La respuesta final es ""racismo, clase"


In [26]:
sample_df.to_csv("../data/sample_dev_df.csv", index=False)

In [27]:
## Error analysis

In [62]:
import pandas as pd

pd.options.display.max_colwidth = 1000

sample_df = pd.read_csv("../data/sample_dev_df.csv")

In [63]:
sample_df[["context_tweet", "text", "text_label", "pred_cot"]]

,context_tweet,text,text_label,pred_cot
0,Coronavirus: un diario inglés asegura que los mercados chinos siguen vendiendo murciélagos y carne de animales domésticos https://t.co/qFPs4aHf6D,@usuario Que gente de mierda .,nada,"El texto no contiene ningún tipo de referencia a características protegidas ni expresiones discriminatorias. La respuesta final es ""nada""."
1,"Murió el periodista Mario Pereyra, ícono de la radio de Córdoba https://t.co/MZtxmqFP4o",@usuario Cuánto zurdito resentido en los comentarios...,política,"El texto utiliza el término ""zurdito"" de manera despectiva para referirse a personas de ideología política izquierdista. La respuesta final es ""política""."
2,"""Se terminó, a salir a la calle”: detuvieron a un hombre que se grabó llamando a romper la cuarentena obligatoria https://t.co/VCK2uezc9p",@usuario Flor de imbécil,nada,"El texto contiene un insulto, pero no se dirige a ninguna característica protegida. La respuesta final es ""nada""."
3,El polémico mensaje de Cinthia Fernández sobre la muerte de un delincuente en un intento de robo https://t.co/odv7PghHxL,@usuario Y que tiene tiene de polémico???,nada,"El texto no contiene discurso discriminatorio. La respuesta final es ""nada""."
4,Coronavirus: China anunció que no registró ningún caso nuevo local https://t.co/bg8d1fSwBX https://t.co/M47osbSpVL,@usuario Chinos de mierda q se maten todos,racismo,"Desea que se maten los chinos por ser los supuestos culpables del COVID-19. La respuesta final es ""racismo"""
...,...,...,...,...
695,"El contundente mensaje de Noelia Marzol sobre el feminismo y el sexo: ""Queremos coger en libertad"" https://t.co/Quo3h68jdU",@usuario Si.lo.hacen,nada,"No se menciona ninguna característica protegida en el texto. La respuesta final es ""nada""."
696,Alberto Fernández: “La pandemia finalmente nos dejó más muertos de los que creíamos tener” https://t.co/0T7hXsGS65,"@usuario baja el dedito , goberna y deja de gritarnos",nada,"El texto contiene una crítica hacia Alberto Fernández, pero no se basa en ninguna de las características protegidas mencionadas. La respuesta final es ""nada""."
697,"Tras las críticas del kirchnerismo duro, Alberto Fernández llamará a Maduro para explicarle la condena argentina en la ONU | Por Román Lejtman https://t.co/yYnNvu50oO","@usuario ""Entendeme, Nicolás. Vos sos un poco brusco con los ciudadanos. Si lo taparas un poco tal vez, pero así de obvio no se puede""",nada,"El texto no contiene discurso discriminatorio, solamente expresa una opinión sobre la forma de actuar de Nicolás Maduro. La respuesta final es ""nada""."
698,Los mapuches avanzaron en la toma de tierras en la Patagonia: cada hectárea está valuada en USD 100 mil https://t.co/52gdoBivK6,"@usuario Que los caguen a balazos y los echen como perros.\nUn tipo que labura todo el día y aporta al Estado no puede adquirir un triste terreno, y si con suerte lo adquiere, tiene que solicitar un crédito o préstamo para poder construir.",racismo,"El texto desea la violencia y discriminación hacia los mapuches por su condición de pertenecer a un pueblo aborigen. Además, se refiere a su condición de ""perros"" y alude a su situación socioeconómica. La respuesta final es ""racismo, clase"


In [64]:
# Check "la respuesta final" is in every pred_cot

sample_df[~sample_df["pred_cot"].str.contains("la respuesta final")]

,id,title,text,context_tweet,HATEFUL,body,CALLS,WOMEN,LGBTI,RACISM,CLASS,POLITICS,DISABLED,APPEARANCE,CRIMINAL,num_hatred,text_label,pred_cot
0,339709,Coronavirus: un diario inglés asegura que los mercados chinos siguen vendiendo murciélagos y carne de animales domésticos,@usuario Que gente de mierda .,Coronavirus: un diario inglés asegura que los mercados chinos siguen vendiendo murciélagos y carne de animales domésticos https://t.co/qFPs4aHf6D,0,"Perros y gatos aterrorizados metidos en jaulas oxidadas. Murciélagos y escorpiones ofrecidos a la venta como medicina tradicional. Conejos y patos sacrificados y desollados uno al lado del otro en un piso de piedra cubierto de sangre, suciedad y restos de animales.\n\nEsas fueron las escenas profundamente inquietantes ayer sábado cuando China celebró su ""victoria"" sobre el coronavirus al reabrir mercados de carne del tipo que inició la pandemia hace tres meses, sin ningún intento aparente de elevar los estándares de higiene para prevenir un brote futuro, afirma el diario inglés Daily Mail.\n\nUn corresponsal de Mail on Sunday observó ayer cómo miles de clientes acudían en masa a un mercado en expansión en Guilin, suroeste de China.\n\nPerros y gatos aterrorizados metidos en jaulas oxidadas.\n\nAllí se apilaron jaulas de diferentes especies una encima de la otra. En otro mercado de carne en Dongguan, en el sur de China, otro corresponsal fotografió a un vendedor de medicamentos que ...",0,0,0,0,0,0,0,0,0,0,nada,"El texto no contiene ningún tipo de referencia a características protegidas ni expresiones discriminatorias. La respuesta final es ""nada""."
1,395003,"Murió el periodista Mario Pereyra, ícono de la radio de Córdoba",@usuario Cuánto zurdito resentido en los comentarios...,"Murió el periodista Mario Pereyra, ícono de la radio de Córdoba https://t.co/MZtxmqFP4o",1,"El periodista Mario Pereyra, una de las grandes leyendas de la radio de Córdoba y una figura preponderante en la radiofonía nacional, murió en la madrugada del domingo a los 77 años, después de no haberse podido recuperar del contagio de coronavirus.\n\nLa triste noticia fue comunicada en las redes sociales por la propia radio Cadena 3, espacio que tuvo a Pereyra como uno de sus creadores y donde se destacó y se convirtió en un ícono de las mañanas cordobesas durante décadas.\n\n""La familia de Cadena 3 Argentina hoy abraza especialmente a Estelita, a todos los familiares de Mario Pereyra y a sus oyentes en el país y el mundo.\n\nMario nos ha dejado un camino: hacer la gran Radio. La radio que entretiene, que informa, que opina y que se ofrece a nuestra gente para que ella también se exprese.\n\nSeguiremos siendo en honor a Mario, la Radio comprometida con la libertad y la República. La radio que acompaña a todos, la radio de la gente.\n\nDescansa en paz querido Mario Pereyra, hasta...",0,0,0,0,0,1,0,0,0,1,política,"El texto utiliza el término ""zurdito"" de manera despectiva para referirse a personas de ideología política izquierdista. La respuesta final es ""política""."
2,381327,"""Se terminó, a salir a la calle"": detuvieron a un hombre que se grabó llamando a romper la cuarentena obligatoria",@usuario Flor de imbécil,"""Se terminó, a salir a la calle”: detuvieron a un hombre que se grabó llamando a romper la cuarentena obligatoria https://t.co/VCK2uezc9p",0,"Arrestan a un hombre en Mar del Plata por dar información falsa de la pandemia\n\nAníbal Raúl Arrarte Artigalas, de 43 años, se apostó en medio del Boulevard Marítimo y la avenida Colón, en Mar del Plata, sacó su celular y comenzó a filmarse. En el video, predicaba que la crisis sanitaria que provocó el coronavirus se había terminado. De barba cuidada, camisa blanca con algunos botones sueltos y una sonrisa, le habló a la sociedad: ""Se terminó. Ya está. A salir a la calle"".\n\nDespués de grabarse, le envió la filmación por Whatsapp a sus conocidos. Inmediatamente, el video se viralizó. A tal punto que le llegó a agentes de la policía bonaerense de esa ci

In [65]:
from rioplatense_hs.preprocessing import text_to_label, labels


pred_labels = [f"PRED_{l}" for l in labels]

for idx, value in sample_df["pred_cot"].items():

    preds = text_to_label(value)

    for k, v in preds.items():
        sample_df.loc[idx, f"PRED_{k}"] = int(v)

In [66]:
from sklearn.metrics import classification_report
gold_labels = sample_df[labels]
preds = sample_df[pred_labels].astype(int)


print(classification_report(gold_labels, preds, target_names=labels))

              precision    recall  f1-score   support

       WOMEN       0.33      0.43      0.38        21
       LGBTI       0.40      0.62      0.48        13
      RACISM       0.55      0.82      0.66        40
       CLASS       0.15      0.40      0.22        10
    POLITICS       0.19      0.59      0.29        22
    DISABLED       0.33      0.20      0.25         5
  APPEARANCE       0.30      0.32      0.31        22
    CRIMINAL       0.24      0.35      0.29        17

   micro avg       0.32      0.54      0.40       150
   macro avg       0.31      0.47      0.36       150
weighted avg       0.35      0.54      0.41       150
 samples avg       0.10      0.10      0.09       150



/home/jmperez/projects/rioplatense_hate_speech/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jmperez/projects/rioplatense_hate_speech/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [70]:
label_hate = sample_df.num_hatred > 0
pred_hate = sample_df[pred_labels].sum(axis=1) > 0

print(classification_report(label_hate, pred_hate))

              precision    recall  f1-score   support

       False       0.92      0.77      0.84       582
        True       0.37      0.66      0.48       118

    accuracy                           0.75       700
   macro avg       0.64      0.72      0.66       700
weighted avg       0.83      0.75      0.78       700



In [72]:
sample_df[pred_hate & ~label_hate][["context_tweet", "text", "text_label", "pred_cot"]].sample(10)

,context_tweet,text,text_label,pred_cot
573,Nancy Pazos: “CFK le saca ochenta cuerpos de ventaja a cualquier político”. | Por Federico Millenaar https://t.co/t0Y6XkYS7z,@usuario Por chorra lo decís???? Jaaa,nada,"El texto descalifica a la persona mencionada por ser ""chorra"", lo cual se refiere a su supuesta condición de criminal. La respuesta final es ""criminal""."
575,China sancionó al frigorífico argentino responsable del envío de carne de exportación en el que se detectó coronavirus | Por Juan Martín Melo\nhttps://t.co/ahsudUye63,"@usuario Jaja, a ver si entienden que no es para nada un paraíso el país de los ojos rasgados",nada,"El texto hace referencia al origen étnico de los chinos de forma peyorativa. La respuesta final es ""racismo""."
100,Las razones de la Justicia para dejar libre al acusado de drogar y violar a la joven venezolana en su primer día de trabajo | Por Leonardo Scannone https://t.co/xeCBm8I0MC,@usuario Qué pasa que el “colectivo feminista” no colgó aún a la jueza en la plaza pública?,nada,"El texto menciona al ""colectivo feminista"" de manera despectiva, insinuando que podrían haber actuado violentamente. La respuesta final es ""mujer""."
176,Coronavirus: las terribles imágenes del mercado donde se originó la pandemia https://t.co/Sy9BvJATax,@usuario Como se llama la obra? Estigmaticemos a los chinos,nada,"El texto alude a estigmatizar a los chinos por ser los supuestos culpables del COVID-19. La respuesta final es ""racismo"
539,Tres extranjeros incumplían la cuarentena total y agredieron a los policías que quisieron identificarlos https://t.co/r40lhweP5P,"@usuario @usuario @usuario cuando se va a comenzar deportar a este tipo de extranjero ni bien se portan mal? Cuidennos en serio, hagamos un país serio.",nada,"El texto alude a la nacionalidad extranjera de las personas y sugiere que se las deporte por portarse mal. La respuesta final es ""racismo"""
438,La historia de la modelo colombiana trans que besa la panza de su esposo embarazado de ocho meses https://t.co/9eSCC1eOHj,@usuario Como cambia todo ahora. Hasta la forma de putear. Imaginen decir. Ándate a la concha de tu padre! O decir. Pero el puto que te parió!! Es un quilombo. Pero hay que aceptarlo,nada,"El texto hace referencia al uso de insultos utilizando términos relacionados a la orientación sexual. La respuesta final es ""lgbti"""
461,🔴 AHORA | Putin anunció que Rusia desarrolló la “primera” vacuna contra el coronavirus https://t.co/XurRJ3FKSx https://t.co/8sgc48u6MR,@usuario A estos y los chinos no les creo nada. Que empiecen por tener democracia primero.,nada,"El texto desestima la credibilidad de los rusos y los chinos por ser de esos países. La respuesta final es ""racismo, política""."
479,Liberaron a un acusado de violación por riesgo de coronavirus: salió de la cárcel y mató a tiros a la víctima https://t.co/YkZWErgoYa,@usuario Debe haber sido en Argentina. Si no lo aclarararían en el título.,nada,"El texto hace referencia a la nacionalidad de Argentina como si estuviera implicando que allí ocurren situaciones de este tipo. La respuesta final es ""racismo""."
647,"Confirman otro procesamiento a Milagro Sala por coima, robo y destrucción de un expediente https://t.co/ODXlYnln27","@usuario chorra presa, solo eso.",nada,"El texto se refiere a la persona como ""chorra"", lo cual alude a su condición de criminal. La respuesta final es ""criminal""."
93,Pablo Moyano amenaza con un paro de recolectores en la Ciudad de Buenos Aires https://t.co/tPArqK6otB https://t.co/8F6nHRi0Jr,@usuario CUANTA PLATA TIENEN ENTRE ESTOS TRES TIPOS. SON MULTIMILLONARIOS.,nada,"El texto menciona que estos tres individuos son multimillonarios, lo cual hace referencia a su situación económica o clase social. La respuesta final es ""clase""."


In [67]:
# LGBTI errors

label = "APPEARANCE"
sample_df.loc[sample_df[label] != sample_df[f"PRED_{label}"]][["context_tweet", "text", "text_label", "pred_cot"]]

,context_tweet,text,text_label,pred_cot
6,"El tamaño, ¿importa?: cómo medirlo y consejos para una penetración más placentera | Por Cecilia Ce https://t.co/6MNdfYk5LI","@usuario Por supuesto que el tamaño NO IMPORTA,,,,,TENERLA CHICA NO CUENTA Y NO IMPORTA😂😂😂😂😂😂😂😂",nada,"El texto alude al tamaño del miembro masculino como algo negativo, lo cual puede ser considerado como discurso discriminatorio hacia los hombres de tener un tamaño considerado ""chico"". La respuesta final es ""aspecto""."
43,Alfredo Cornejo: “El peronismo de hoy representa al sector parasitario que vive de quienes crean riqueza” https://t.co/p3bKfwF93N,@usuario Viejo bldo,apariencia,"No hay discurso discriminatorio en el mensaje. La respuesta final es ""nada""."
46,Graciela Camaño: “Me parece horrible que el abogado de Cristina Kirchner integre el consejo para la reforma judicial” https://t.co/A9DGv1ktP7,@usuario Pero después vas y votas por su impunidad vieja forra @usuario,apariencia,"El texto utiliza un insulto hacia la persona mencionada, pero no hace referencia a ninguna característica protegida. La respuesta final es ""nada""."
49,"💥[OTRA INDIRECTA]💥 ¿More Rial cuestionó las habilidades de su ex en la cama? &gt;&gt;&gt; Luego de su separación con Facundo Ambrosioni, la mediática hizo una fuerte referencia a su vida sexual. ¡Mirá! https://t.co/BGvJn4JoFo",@usuario Esta acumulando mas leche que mastellone,"mujer, apariencia","El texto hace una referencia a la cantidad de semen que acumula la persona mencionada. No alude a ninguna de las características protegidas. La respuesta final es ""nada""."
74,El líder piquetero Luis D’Elía pidió el indulto presidencial https://t.co/nobQwvkLa4,@usuario Gordo hijo de una gran puta,apariencia,"Es un insulto y no menciona nada relacionado a las características protegidas. La respuesta final es ""nada"""
102,"Ofelia Fernández, contra Macri por su carta: ""Qué gratis es la hipocresía "" https://t.co/YiuZlMWilY https://t.co/il6MNE1w8a","@usuario Totalmente @usuario tenes razón. VOS SOS LA HIPOCRITA NUMERO UNO, cuando dijiste que no ibas a cobrsrtu sueldo negra sucia","mujer, racismo, apariencia","El texto alude a su color de piel y la desprecia mediante insultos racistas. La respuesta final es ""mujer, racismo"""
118,El papa Francisco criticó a la gente que se va de vacaciones para evadir las cuarentenas https://t.co/F0wNI7cWmk,@usuario Re GAGA,"discapacidad, apariencia","El texto no contiene ningún tipo de discurso discriminatorio. La respuesta final es ""nada""."
124,"El tamaño, ¿importa?: cómo medirlo y consejos para una penetración más placentera | Por Cecilia Ce https://t.co/6MNdfYk5LI",@usuario Jaja parece que le mete hasta los huevos. Lo mide desde ahí en la foto. Que notita navideña,nada,"El texto hace un comentario sexualmente inapropiado y hace referencia a los genitales del individuo en cuestión. La respuesta final es ""aspecto""."
131,"Juan José Sebreli sobre la cuarentena: ""Promuevo algo perfectamente democrático, la desobediencia civil"" https://t.co/GckW5bbJrS https://t.co/aQCesuaTiW",@usuario Anda a chupar pasamanos de colectivos \nViejo Gaga!!,"discapacidad, apariencia","Es un insulto y no menciona nada relacionado a las características protegidas. La respuesta final es ""nada"""
142,🔥[¡CON DIGNITY!]🔥 ¡Calu Rivero se desnudó para Instagram! &gt;&gt;&gt; La actriz revolucionó las redes sociales con una foto sin ropa y al borde de la censura. ¡Entrá a la nota y mirá! https://t.co/kyBBH3N2t6,@usuario Quien está desnutrida ???,nada,"El texto se refiere al aspecto físico de la persona y puede ser considerado como una ofensa. La respuesta final es ""aspecto""."


In [39]:
sample_df

,id,title,text,context_tweet,HATEFUL,body,CALLS,WOMEN,LGBTI,RACISM,...,text_label,pred_cot,PRED_WOMEN,PRED_LGBTI,PRED_RACISM,PRED_CLASS,PRED_POLITICS,PRED_DISABLED,PRED_APPEARANCE,PRED_CRIMINAL
0,339709,Coronavirus: un diario inglés asegura que los mercados chinos siguen vendiendo murciélagos y carne de animales domésticos,@usuario Que gente de mierda .,Coronavirus: un diario inglés asegura que los mercados chinos siguen vendiendo murciélagos y carne de animales domésticos https://t.co/qFPs4aHf6D,0,"Perros y gatos aterrorizados metidos en jaulas oxidadas. Murciélagos y escorpiones ofrecidos a la venta como medicina tradicional. Conejos y patos sacrificados y desollados uno al lado del otro en un piso de piedra cubierto de sangre, suciedad y restos de animales.\n\nEsas fueron las escenas profundamente inquietantes ayer sábado cuando China celebró su ""victoria"" sobre el coronavirus al reabrir mercados de carne del tipo que inició la pandemia hace tres meses, sin ningún intento aparente de elevar los estándares de higiene para prevenir un brote futuro, afirma el diario inglés Daily Mail.\n\nUn corresponsal de Mail on Sunday observó ayer cómo miles de clientes acudían en masa a un mercado en expansión en Guilin, suroeste de China.\n\nPerros y gatos aterrorizados metidos en jaulas oxidadas.\n\nAllí se apilaron jaulas de diferentes especies una encima de la otra. En otro mercado de carne en Dongguan, en el sur de China, otro corresponsal fotografió a un vendedor de medicamentos que ...",0,0,0,0,...,nada,"El texto no contiene ningún tipo de referencia a características protegidas ni expresiones discriminatorias. La respuesta final es ""nada"".",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,395003,"Murió el periodista Mario Pereyra, ícono de la radio de Córdoba",@usuario Cuánto zurdito resentido en los comentarios...,"Murió el periodista Mario Pereyra, ícono de la radio de Córdoba https://t.co/MZtxmqFP4o",1,"El periodista Mario Pereyra, una de las grandes leyendas de la radio de Córdoba y una figura preponderante en la radiofonía nacional, murió en la madrugada del domingo a los 77 años, después de no haberse podido recuperar del contagio de coronavirus.\n\nLa triste noticia fue comunicada en las redes sociales por la propia radio Cadena 3, espacio que tuvo a Pereyra como uno de sus creadores y donde se destacó y se convirtió en un ícono de las mañanas cordobesas durante décadas.\n\n""La familia de Cadena 3 Argentina hoy abraza especialmente a Estelita, a todos los familiares de Mario Pereyra y a sus oyentes en el país y el mundo.\n\nMario nos ha dejado un camino: hacer la gran Radio. La radio que entretiene, que informa, que opina y que se ofrece a nuestra gente para que ella también se exprese.\n\nSeguiremos siendo en honor a Mario, la Radio comprometida con la libertad y la República. La radio que acompaña a todos, la radio de la gente.\n\nDescansa en paz querido Mario Pereyra, hasta...",0,0,0,0,...,política,"El texto utiliza el término ""zurdito"" de manera despectiva para referirse a personas de ideología política izquierdista. La respuesta final es ""política"".",0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,381327,"""Se terminó, a salir a la calle"": detuvieron a un hombre que se grabó llamando a romper la cuarentena obligatoria",@usuario Flor de imbécil,"""Se terminó, a salir a la calle”: detuvieron a un hombre que se grabó llamando a romper la cuarentena obligatoria https://t.co/VCK2uezc9p",0,"Arrestan a un hombre en Mar del Plata por dar información falsa de la pandemia\n\nAníbal Raúl Arrarte Artigalas, de 43 años, se apostó en medio del Boulevard Marítimo y la avenida Colón, en Mar del Plata, sacó su celular y comenzó a filmarse. En el video, predicaba que la crisis sanitaria que provocó el coronavirus se había terminado. De barba cuidada, camisa blanca con algunos botones sueltos y una sonrisa, le habló a la sociedad: ""Se terminó. Ya está. A salir a la calle"".\n\nDespués de grabarse, le envió la filmación por Whatsapp a sus conocidos. Inmedia